In [37]:
from natasha import MorphVocab
import sqlite3
import pandas as pd

morph_vocab = MorphVocab()

def lemmatization(token):
    forms = morph_vocab(token)
    forms_lst = []
    for f in forms:
        forms_lst.append(list(f)[2])
    forms_lst = list(set(forms_lst))
    return forms_lst

In [45]:
import json

In [38]:
con = sqlite3.connect('telegram1.db')

In [39]:
c = con.cursor()
con.commit()

In [98]:
def get_posts_for_token(qua: str):
    x = (qua, )
    
    list_id_posts = []
    for row in c.execute("""SELECT list_posts_id FROM tokens
                            WHERE token=?""", x):
        list_id_posts.extend(row[-1].replace('[', '').replace(']', '').replace(' ', '').split(','))
 
    channel_names = []
    post_texts = []
    for id_p in list_id_posts:
        z = (int(id_p), )
        for row in c.execute("""SELECT channels.name, posts.text FROM posts JOIN channels
            ON posts.id_channel = channels.id_channel WHERE id_post=?""", z):
            channel_names.append(row[-2])
            post_texts.append(row[-1])
    return post_texts, channel_names

In [99]:
def search_lemma(q):
    x = (q, )
    for row in c.execute("""SELECT id_token, lemmas.list_tokens FROM tokens JOIN lemmas
        ON lemmas.id_lemma = tokens.id_lemma WHERE token=?""", x):
        tokens_id = row[-1].replace('[', '').replace(']', '').replace(' ', '').split(',')
        i = str(row[-2])
        tokens_id.remove(i)

    list_tokens = []
    for j in tokens_id:
        y = (j, )
        for row in c.execute("""SELECT token FROM tokens
                                WHERE id_token=?""", y):
            list_tokens.append(row[-1])

    post_texts = []
    channel_names = []
    for t in list_tokens:
        a, b = get_posts_for_token(t)
        post_texts.extend(a)
        channel_names.extend(b)
    return post_texts, channel_names

In [108]:
def search_synonyms(q):
    x = (q, )
    for row in c.execute("""SELECT list_synonyms FROM tokens
        WHERE token=?""", x):
        synonyms_id = row[-1]

    list_synonyms = []
    for j in synonyms_id:
        y = (j, )
        for row in c.execute("""SELECT token FROM tokens
                                WHERE id_token=?""", y):
            list_synonyms.append(row[-1])

    post_texts = []
    channel_names = []
    for s in list_synonyms:
        a, b = get_posts_for_token(s)
        post_texts.extend(a)
        channel_names.extend(b)
    return post_texts, channel_names

In [109]:
def search(qua: str):
    q = qua.lower()
    post_texts, channel_names = get_posts_for_token(q)
    if len(post_texts) == 0:
        with open('posts.json', 'a', encoding='utf-8') as file:
            json.dump('No posts yet...', file)
    else:    
        a, b = search_lemma(q)
        post_texts.extend(a)
        channel_names.extend(b)
        c, d = search_synonyms(q)
        post_texts.extend(c)
        channel_names.extend(d)
        df_output = pd.DataFrame()
        df_output['post'] = post_texts
        df_output['channel'] = channel_names
        output = df_output.to_dict('records')
        with open('posts.json', 'a', encoding='utf-8') as file:
            json.dump(output, file)

In [110]:
search('делаем')

['сегодня делаем лол', 'сегодня делаем мем', 'сегодня делаем руссику'] ['lol', 'lol', 'russica2']
---------
['сегодня делаем лол', 'сегодня делаем мем', 'сегодня делаем руссику'] ['lol', 'lol', 'russica2']
[2]
['делаем']
---------
['сегодня делаем лол', 'сегодня делаем мем', 'сегодня делаем руссику', 'сегодня делаем лол', 'сегодня делаем мем', 'сегодня делаем руссику'] ['lol', 'lol', 'russica2', 'lol', 'lol', 'russica2']
                     post   channel
0      сегодня делаем лол       lol
1      сегодня делаем мем       lol
2  сегодня делаем руссику  russica2
3      сегодня делаем лол       lol
4      сегодня делаем мем       lol
5  сегодня делаем руссику  russica2
